In [ ]:
import pandas as pd
import numpy as np

PIPELINE = '2023'

df = pd.read_excel("Bewertungen.xlsx", sheet_name=PIPELINE)
df['Startup'] = df['Startup'].str.split(" -").str.get(0)
df.head()
len(df)

In [ ]:
import os
dir = "data/FinTech Germany Award " + PIPELINE
try:
    os.mkdir(dir)
except:
    print(dir + " already exists")
print(dir)

In [ ]:
import pandas as pd
from fuzzywuzzy import process
import os

df1 = pd.read_excel(dir + "/general.xlsx")
df1 = df1[["Startup/Person Name", "Startup ID"]]

# Function to find the best match using fuzzy matching
def fuzzy_match(target, choices, threshold=80):
    match, score = process.extractOne(target, choices)
    if score >= threshold:
        return match
    return None

# Apply fuzzy matching to df1['SearchKey'] against all df2['Startup']
df['BestMatch'] = df['Startup'].apply(
    lambda x: fuzzy_match(x, df1['Startup/Person Name'].tolist())
)

# Merge DataFrames based on the fuzzy match result
dfa = df.merge(df1, left_on='BestMatch', right_on='Startup/Person Name', how='left')
dfa.drop(["BestMatch"], axis=1, inplace=True)
df = dfa
df.head()

In [ ]:
# Replace NaN values with 4
ratings_df = df.iloc[:, 1:]

ratings_df = ratings_df.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with 4
ratings_df.fillna(4, inplace=True)

## Round down numbers to integers
ratings_df = np.floor(ratings_df)

## Replace values greater than 7 with 7
ratings_df = np.where(ratings_df > 7, 7, ratings_df)
ratings_df = np.where(ratings_df < 1, 1, ratings_df)

pd.DataFrame(ratings_df).head()

In [ ]:
# Calculate Max Score
df['Max Score'] = ratings_df.max(axis=1)

# Calculate Min Score
df['Min Score'] = ratings_df.min(axis=1)

# Calculate Number of "4" ratings
df['Number of "4" ratings'] = np.count_nonzero(ratings_df == 4, axis=1)

# Calculate Average evaluation
df['Average evaluation'] = np.nanmean(ratings_df, axis=1)

import numpy.ma as ma

masked_ratings = ma.masked_equal(ratings_df, 4)
df['Average evaluation (excluding 4)'] = ma.mean(masked_ratings, axis=1).filled(np.nan)

masked_ratings = ma.masked_equal(masked_ratings, 3)
masked_ratings = ma.masked_equal(masked_ratings, 5)
df['Average evaluation (excluding 3, 4 and 5)'] = ma.mean(masked_ratings, axis=1).filled(np.nan)

In [ ]:
df.to_excel(dir + '/scores.xlsx', index=False)